In [2]:
from requests import Session
from lxml import html
from datetime import datetime
import time
import random
import sys
import logging
import psycopg2
#rds settings
rds_host  = "database-spotifier.c5eevkz7wazj.us-east-2.rds.amazonaws.com"
name = "tally_ds" 
db_name = "postgres" 
password = "P@ssw0rd"


logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = psycopg2.connect(host=rds_host, user=name, password=password, database=db_name, port=5432, connect_timeout=5)
except psycopg2.OperationalError as e:
    logger.error("ERROR: Unexpected error: Could not connect to postGreSQL instance.")
    logger.error(e)
    sys.exit()

logger.info("SUCCESS: Connection to RDS postGreSQL instance succeeded")


def yelpScraper(business_id):
    base_url = "https://www.yelp.com/biz/" # add business id
    api_url = "/review_feed?sort_by=date_desc&start=" # add number
    results = []
    for n in range(1):
        with Session() as s:
            url = base_url + business_id + api_url + str(n*20)    
            with s.get(url, timeout=5) as r:    
                if r.status_code==200:
                    response = dict(r.json()) 
                    _html = html.fromstring(response['review_list']) 
                    dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                    dates = [datetime.strptime(d.strip(), format("%m/%d/%Y")) for d in dates]
                    stars = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")
                    stars = [float(s.split(' ')[0]) for s in stars]
                    texts = [e.text for e in _html.xpath("//div[@class='review-content']/p")]
                    results = results + [[date, star, text] for date, star, text in zip(dates, stars, texts)]
        time.sleep(random.uniform(0.1, 0.5))
    return results
def dbConnect(results):
    item_count = 0
    with conn.cursor() as cur:
        for i in results:
            x = i[0]
            y = i[1]
            z = i[2][:18]#
            cur.execute("create table lab.review ( uuid NOT NULL DEFAULT uuid_generate_v4(), review_id character varying COLLATE pg_catalog."default", business_id character varying COLLATE pg_catalog."default", user_id character varying COLLATE pg_catalog."default", stars double precision, datetime timestamp without time zone, date date, "time" time without time zone, text character varying COLLATE pg_catalog."default", "timestamp" timestamp with time zone, data_source smallint)")
#             cur.execute('INSERT INTO lab.receipt (receipt_date, origin, destination) VALUES (%s, %s, %s)', (x, y, z))

            conn.commit()
            cur.execute("select * from lab.receipt")
            for row in cur:
                item_count += 1
                logger.info(row)
        conn.commit()
    return "Added %d items from RDS postGreSQL table" %(item_count)
    
# def lambda_handler(event, context):
#     yelpScraper("jga_2HO_j4I7tSYf5cCEnQ")
#     dbConnect()
#     return "it worked"

results = yelpScraper("jga_2HO_j4I7tSYf5cCEnQ")
dbConnect(results)

'Added 2950 items from RDS postGreSQL table'

In [10]:
import sys
import logging
import psycopg2
#rds settings
rds_host  = "database-spotifier.c5eevkz7wazj.us-east-2.rds.amazonaws.com"
name = "tally_ds" 
db_name = "postgres" 
password = "P@ssw0rd"


logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = psycopg2.connect(host=rds_host, user=name, password=password, database=db_name, port=5432, connect_timeout=5)
except psycopg2.OperationalError as e:
    logger.error("ERROR: Unexpected error: Could not connect to postGreSQL instance.")
    logger.error(e)
    sys.exit()

logger.info("SUCCESS: Connection to RDS postGreSQL instance succeeded")

def dbConnect():
    item_count = 0

    with conn.cursor() as cur:
        cur.execute('create table lab.review (uuid uuid NOT NULL DEFAULT uuid_generate_v4(), review_id character varying COLLATE pg_catalog."default", business_id character varying COLLATE pg_catalog."default", user_id character varying COLLATE pg_catalog."default", stars double precision, datetime timestamp without time zone, date date, time time without time zone, text character varying COLLATE pg_catalog."default", timestamp timestamp with time zone, data_source smallint)')
        conn.commit()
        cur.execute("select * from lab.receipt")
        for row in cur:
            item_count += 1
            logger.info(row)
            #print(row)
    conn.commit()

    return "Added %d items from RDS postGreSQL table" %(item_count)
    
# def lambda_handler(event, context):
#     dbConnect()
#     return "it worked"

dbConnect()


'Added 158 items from RDS postGreSQL table'